In [1]:
from bs4 import BeautifulSoup
import time
import requests
html_doc=requests.get('https://kidsbrandstore.se').content.decode()
soup = BeautifulSoup(html_doc, 'html.parser')
links=soup.find("div",attrs={"id":"bottom-types"})

categorys=[]
main_urls=[]
for link in links.find_all("a"):
    main_urls.append('https://kidsbrandstore.se'+link['href'])
    categorys.append(link.text.strip())
categorys

['BYXOR',
 'JEANS',
 'JUMPSUIT/BYXDRESS',
 'KLäNNINGAR/KJOLAR',
 'SHORTS',
 'STRUMPBYXOR/LEGGINGS',
 'STRUMPOR/SOCKOR',
 'UNDERKLäDER',
 'HALSDUKAR/SCARFS',
 'JACKOR/VäSTAR',
 'REGNKLäDER',
 'TäCKBYXOR/SKALBYXOR',
 'UNDERSTäLL',
 'HUVTRöJOR',
 'PIKE/RUGBYTRöJOR',
 'SET/TRäNINGSOVERALLER',
 'SKJORTOR',
 'T-SHIRT/LINNEN',
 'TOPPAR/BLUSAR',
 'TRöJOR/CARDIGANS',
 'BADKLäDER/BADROCKAR',
 'HANDDUKAR',
 'KEPSAR',
 'KLOCKOR',
 'MöSSOR',
 'PRESENTFöRPACKNINGAR',
 'PRESENTKORT',
 'SANDALER',
 'SKäRP/BäLTEN',
 'VANTAR',
 'VäSKOR/NECESSäRER']

In [2]:
main_urls

['https://kidsbrandstore.se/byxor/',
 'https://kidsbrandstore.se/jeans/',
 'https://kidsbrandstore.se/jumpsuit-byxdress/',
 'https://kidsbrandstore.se/klanningar-kjolar/',
 'https://kidsbrandstore.se/shorts/',
 'https://kidsbrandstore.se/strumpbyxor-leggings/',
 'https://kidsbrandstore.se/strumpor-sockor/',
 'https://kidsbrandstore.se/underklader/',
 'https://kidsbrandstore.se/halsdukar-scarfs/',
 'https://kidsbrandstore.se/jackor-vastar/',
 'https://kidsbrandstore.se/regnklader/',
 'https://kidsbrandstore.se/tackbyxor-skalbyxor/',
 'https://kidsbrandstore.se/understall/',
 'https://kidsbrandstore.se/huvtrojor/',
 'https://kidsbrandstore.se/pike-rugbytrojor/',
 'https://kidsbrandstore.se/set-traningsoveraller/',
 'https://kidsbrandstore.se/skjortor/',
 'https://kidsbrandstore.se/t-shirt-linnen/',
 'https://kidsbrandstore.se/toppar-blusar/',
 'https://kidsbrandstore.se/trojor-cardigans/',
 'https://kidsbrandstore.se/badklader-badrockar/',
 'https://kidsbrandstore.se/handdukar/',
 'https

In [3]:

from google.cloud import storage

def upload_to_bucket(blob_name, path_to_file):
    """ Upload data to a bucket""" #doc string in function

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'auth.json')

    #print(buckets = list(storage_client.list_buckets())

    bucket = storage_client.get_bucket('urbankiddo')
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(path_to_file)

    #returns a public url
    return blob.public_url

In [4]:
import csv


f = open("kidsbrandstore.csv", "w", encoding='utf-8')
writer = csv.DictWriter(f, fieldnames=["name","brand","gender","category","size","price","url","img_url","article_no","blob_url"])
writer.writeheader()
f.close()

import pandas as pd
df=pd.read_csv("kidsbrandstore.csv", encoding='utf-8')
df.head()

,name,brand,gender,category,size,price,url,img_url,article_no,blob_url


In [5]:
!mkdir kidsbrandstore

In [6]:
gender='boy'
for c,main_url in enumerate(main_urls):
    category=categorys[c]
    gender_url=main_url+'20/760'    
    html_doc2=requests.get(gender_url).content.decode()
    soup2 = BeautifulSoup(html_doc2, 'html.parser')
    pages=soup2.find("div",attrs={"class":"paginationControl"})
    page_urls=[gender_url]
    if type(pages)==type(None):
        print('None type no pages')
        pass
    else:
        page_urls.append(gender_url+pages.find("a")['href'])
    print(page_urls)
    for page_url in page_urls:
        html_doc3=requests.get(page_url).content.decode()
        soup3 = BeautifulSoup(html_doc3, 'html.parser')
        for u in soup3.find_all("a",attrs={"class":"product-holder productIsActive"}):
            url='https://kidsbrandstore.se'+u['href']
            article_no=u['id']
            name=u.find("span",attrs={"class":"product-name-holder"}).text
            brand=u.find("span",attrs={"class":"product-brand-name-holder"}).text
            price=u.find("span",attrs={"class":"product-price"}).text
            img_url=u.find("img")['data-src']
            print(url)
            
            ac='kidsbrandstore'+'/'+article_no+str('.jpg')
            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

            blob_name=str('kidsbrandstore')+str('/')+str(article_no)
            path_to_file='kidsbrandstore/'+article_no+str('.jpg')
            blob_url=upload_to_bucket(blob_name, path_to_file)

            
            
            html_doc3=requests.get(url).content.decode()
            soup3 = BeautifulSoup(html_doc3, 'html.parser')
            sizes=soup3.find("div",attrs={"id":"attribute-select-holder"})
            for sz in sizes.find_all("label"):
                size=sz.text
                with open("kidsbrandstore.csv", "a", encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow([name,brand,gender,category,size,price,url,img_url,article_no,blob_url])

        
        
    

None type no pages
['https://kidsbrandstore.se/byxor/20/760']
https://kidsbrandstore.se/tommy-hilfiger-essential-scanton-slim-35919
https://kidsbrandstore.se/under-armour-eu-cotton-fleece-jogger-34729
https://kidsbrandstore.se/under-armour-prototype-pant-34691
https://kidsbrandstore.se/replay-pants-37525
https://kidsbrandstore.se/replay-pants-37524
https://kidsbrandstore.se/peak-performance-jr-orig-pa-36585
https://kidsbrandstore.se/peak-performance-jr-orig-pa-36582
https://kidsbrandstore.se/calvin-klein-slim-cotton-chino-35853
https://kidsbrandstore.se/gant-gant-logo-sweat-pants-35774
https://kidsbrandstore.se/gant-gant-logo-sweat-pants-35773
https://kidsbrandstore.se/scotch-soda-club-nomade-sweat-pant-35534
https://kidsbrandstore.se/scotch-soda-club-nomade-sweat-pant-35533
https://kidsbrandstore.se/calvin-klein-neon-tape-sweatpants-35855
https://kidsbrandstore.se/lacoste-pant-33464
https://kidsbrandstore.se/ralph-lauren-belted-pant-bottoms-pant-32998
https://kidsbrandstore.se/ralph-l

https://kidsbrandstore.se/tommy-hilfiger-quickdry-surf-shorts-35950
https://kidsbrandstore.se/calvin-klein-monogram-short-boy-mid-blue-35867
https://kidsbrandstore.se/frank-dandy-kids-solid-sweat-shorts-35484
https://kidsbrandstore.se/henri-lloyd-lb-sweat-shorts-34962
https://kidsbrandstore.se/henri-lloyd-garment-dyed-chino-short-34961
https://kidsbrandstore.se/henri-lloyd-garment-dyed-chino-short-34960
https://kidsbrandstore.se/grunt-chino-twill-shorts-34644
https://kidsbrandstore.se/grunt-chino-twill-shorts-34643
https://kidsbrandstore.se/grunt-space-dk-grey-shorts-34627
https://kidsbrandstore.se/grunt-space-black-shorts-34626
https://kidsbrandstore.se/levis-bermuda-511-33797
https://kidsbrandstore.se/tommy-hilfiger-essential-dobby-belted-chino-33497
https://kidsbrandstore.se/under-armour-match-play-20-golf-short-34739
https://kidsbrandstore.se/peak-performance-jr-orig-sh-33415
https://kidsbrandstore.se/peak-performance-jr-orig-sh-33414
https://kidsbrandstore.se/gant-tb-gant-chino-sh

https://kidsbrandstore.se/lyle-scott-funnel-neck-jacket-34410
https://kidsbrandstore.se/lyle-scott-bomber-jacket-34399
https://kidsbrandstore.se/8848-altitude-liam-jr-liner-jacket-34156
https://kidsbrandstore.se/lyle-scott-outerwear-15897
https://kidsbrandstore.se/barbour-bintl-boys-grange-parka-34125
https://kidsbrandstore.se/gant-the-club-blazer-33365
https://kidsbrandstore.se/sail-racing-jr-ocean-gtx-lumber-33243
https://kidsbrandstore.se/sail-racing-jr-ocean-gtx-lumber-33242
https://kidsbrandstore.se/barbour-b-intl-delta-jacket-25580
https://kidsbrandstore.se/napapijri-k-acalmar-2-34818
https://kidsbrandstore.se/napapijri-k-acalmar-2-34817
https://kidsbrandstore.se/8848-altitude-toby-jr-jacket-34158
https://kidsbrandstore.se/8848-altitude-toby-jr-jacket-34157
https://kidsbrandstore.se/8848-altitude-harrison-jr-softshel-34148
https://kidsbrandstore.se/8848-altitude-harrison-jr-softshel-34146
https://kidsbrandstore.se/8848-altitude-harrison-jr-softshel-34145
https://kidsbrandstore.se

https://kidsbrandstore.se/sail-racing-jr-bowman-hood-33253
https://kidsbrandstore.se/sail-racing-jr-bowman-hood-33252
https://kidsbrandstore.se/sail-racing-jr-bowman-hood-33245
https://kidsbrandstore.se/sail-racing-jr-bowman-zip-hood-33234
https://kidsbrandstore.se/sail-racing-jr-bowman-zip-hood-33228
https://kidsbrandstore.se/sail-racing-jr-bowman-zip-hood-33227
https://kidsbrandstore.se/sail-racing-jr-bowman-zip-hood-33226
https://kidsbrandstore.se/barbour-ess-hoody-36547
https://kidsbrandstore.se/barbour-ess-hoody-36546
https://kidsbrandstore.se/gant-gant-chest-stripe-hr-35756
https://kidsbrandstore.se/gant-collegiate-gant-sweat-hoodie-35772
https://kidsbrandstore.se/gant-collegiate-gant-sweat-hoodie-35771
https://kidsbrandstore.se/gant-collegiate-gant-sweat-hoodie-35747
https://kidsbrandstore.se/gant-collegiate-gant-sweat-hoodie-35744
https://kidsbrandstore.se/franklin-marshall-badge-logo-zip-hoodie-34865
https://kidsbrandstore.se/franklin-marshall-colour-block-zip-hoodie-34857
htt

https://kidsbrandstore.se/gant-tb-gant-full-zip-sweat-hoodie-33332
https://kidsbrandstore.se/scotch-soda-club-nomade-hoodie-32606
https://kidsbrandstore.se/scotch-soda-hoody-with-artworks-34454
https://kidsbrandstore.se/scotch-soda-lightweight-twisted-hood-with-back-artwork-34453
https://kidsbrandstore.se/scotch-soda-zip-through-hoody-with-tape-detail-34452
https://kidsbrandstore.se/gant-tb-the-org-sweat-hoodie-30447
https://kidsbrandstore.se/gant-the-original-full-zip-sweat-hoodie-30363
https://kidsbrandstore.se/napapijri-k-billo-34815
https://kidsbrandstore.se/napapijri-k-billo-34814
https://kidsbrandstore.se/grunt-colin-sweat-cardigan-34638
https://kidsbrandstore.se/levis-sweat-batsweat-24619
https://kidsbrandstore.se/franklin-marshall-badge-logo-zip-hoodie-30683
https://kidsbrandstore.se/franklin-marshall-badge-logo-zip-hoodie-29287
https://kidsbrandstore.se/franklin-marshall-badge-logo-zip-hoodie-29286
https://kidsbrandstore.se/franklin-marshall-f-and-m-hoodie-29285
https://kidsbr

https://kidsbrandstore.se/diadora-hd-suit-5-palle-35601
https://kidsbrandstore.se/diadora-hd-suit-5-palle-35599
https://kidsbrandstore.se/diadora-fz-suit-5-palle-35596
https://kidsbrandstore.se/diadora-fz-suit-5-palle-35593
https://kidsbrandstore.se/champion-full-zip-suit-33856
None type no pages
['https://kidsbrandstore.se/skjortor/20/760']
https://kidsbrandstore.se/tommy-hilfiger-seersucker-shirt-ss-35968
https://kidsbrandstore.se/tommy-hilfiger-oxford-stripe-shirt-ls-35933
https://kidsbrandstore.se/tommy-hilfiger-oxford-stripe-shirt-ls-35932
https://kidsbrandstore.se/ralph-lauren-ls-bd-tops-shirt-32897
https://kidsbrandstore.se/ralph-lauren-ss-bd-tops-knit-32908
https://kidsbrandstore.se/ralph-lauren-ss-bd-tops-knit-32907
https://kidsbrandstore.se/ralph-lauren-ls-bd-tops-shirt-32898
https://kidsbrandstore.se/ralph-lauren-ls-bd-tops-shirt-32896
https://kidsbrandstore.se/lyle-scott-oxford-shirt-ls-34359
https://kidsbrandstore.se/calvin-klein-chambray-regular-shirt-35856
https://kidsbr

https://kidsbrandstore.se/ralph-lauren-crewneck-t-shirt-33016
https://kidsbrandstore.se/ralph-lauren-crewneck-t-shirt-33015
https://kidsbrandstore.se/ralph-lauren-crewneck-t-shirt-33014
https://kidsbrandstore.se/ralph-lauren-crewneck-t-shirt-33001
https://kidsbrandstore.se/scotch-soda-club-nomade-print-tee-35543
https://kidsbrandstore.se/scotch-soda-club-nomade-print-tee-35542
https://kidsbrandstore.se/tommy-hilfiger-original-cn-tee-ss-35954
https://kidsbrandstore.se/tommy-hilfiger-puff-print-tee-ss-35949
https://kidsbrandstore.se/tommy-hilfiger-logo-tee-ss-35945
https://kidsbrandstore.se/tommy-hilfiger-puff-print-tee-ss-35942
https://kidsbrandstore.se/wesc-wesc-t-shirt-jr-35025
https://kidsbrandstore.se/wesc-wesc-t-shirt-jr-35024
https://kidsbrandstore.se/wesc-box-icon-t-shirt-jr-35022
https://kidsbrandstore.se/wesc-yarn-dye-stripe-t-shirt-jr-35021
https://kidsbrandstore.se/wesc-yarn-dye-stripe-t-shirt-jr-35020
https://kidsbrandstore.se/wesc-overlay-t-shirt-jr-35019
https://kidsbrands

https://kidsbrandstore.se/us-polo-assn-funnel-sweat-34932
https://kidsbrandstore.se/ben-sherman-script-bb-sweat-crew-34873
https://kidsbrandstore.se/ben-sherman-funnel-bb-sweat-zip-through-34872
https://kidsbrandstore.se/ben-sherman-funnel-bb-sweat-zip-through-34871
https://kidsbrandstore.se/franklin-marshall-crested-block-oversize-crew-34861
https://kidsbrandstore.se/lyle-scott-logo-crew-lb-34376
https://kidsbrandstore.se/lyle-scott-logo-crew-lb-34374
https://kidsbrandstore.se/calvin-klein-neon-tape-zip-terry-jacket-35824
https://kidsbrandstore.se/calvin-klein-logo-patch-terry-sweatshirt-35823
https://kidsbrandstore.se/replay-jumper-33903
https://kidsbrandstore.se/converse-stacked-wordmark-ft-crew-neck-35055
https://kidsbrandstore.se/replay-jumper-33905
https://kidsbrandstore.se/scotch-soda-basic-scotch-soda-sweat-35535
https://kidsbrandstore.se/tommy-hilfiger-essential-flag-crew-35928
https://kidsbrandstore.se/scotch-soda-basic-scotch-soda-sweat-35536
https://kidsbrandstore.se/wesc-b

https://kidsbrandstore.se/gant-tb-gant-1949-striped-crew-33370
https://kidsbrandstore.se/gant-d1-lt-wt-cotton-crew-33368
https://kidsbrandstore.se/gant-tb-gant-east-coast-c-neck-sweat-33327
None type no pages
['https://kidsbrandstore.se/badklader-badrockar/20/760']
https://kidsbrandstore.se/ralph-lauren-traveler-sh-swimwear-boardshort-32964
https://kidsbrandstore.se/lyle-scott-hibiscus-swimshort-34402
https://kidsbrandstore.se/lyle-scott-taped-swimshort-34386
https://kidsbrandstore.se/lyle-scott-classic-swimshort-34345
https://kidsbrandstore.se/lyle-scott-classic-swimshort-34341
https://kidsbrandstore.se/lyle-scott-classic-swimshort-34339
https://kidsbrandstore.se/ralph-lauren-traveler-swimwear-boxer-32913
https://kidsbrandstore.se/ralph-lauren-traveler-swimwear-boxer-32904
https://kidsbrandstore.se/lyle-scott-classic-swim-shorts-30625
https://kidsbrandstore.se/lyle-scott-classic-swimshort-34340
https://kidsbrandstore.se/lyle-scott-classic-swimshort-34338
https://kidsbrandstore.se/lyle

https://kidsbrandstore.se/lyle-scott-core-backpack-30671
https://kidsbrandstore.se/adidas-originals-nmd-p-black-35115
https://kidsbrandstore.se/eastpak-springer-36999
https://kidsbrandstore.se/eastpak-springer-36998
https://kidsbrandstore.se/eastpak-buddy-36997
https://kidsbrandstore.se/eastpak-padded-pakr-36996
https://kidsbrandstore.se/eastpak-springer-36995
https://kidsbrandstore.se/eastpak-buddy-36994
https://kidsbrandstore.se/eastpak-springer-36988
https://kidsbrandstore.se/tommy-hilfiger-tjm-cool-city-backpack-35900
https://kidsbrandstore.se/calvin-klein-sport-essential-cp-backpack-35-35872
https://kidsbrandstore.se/tommy-hilfiger-sport-mix-washbag-corp-35913
https://kidsbrandstore.se/tommy-hilfiger-corporate-backpack-35898
https://kidsbrandstore.se/calvin-klein-sport-essential-cp-backpack-35-35873
https://kidsbrandstore.se/ralph-lauren-md-pp-tote-tote-medium-32968
https://kidsbrandstore.se/franklin-marshall-franklin-blocked-back-pack-34864
https://kidsbrandstore.se/lyle-scott-co

In [7]:
gender='girl'
for c,main_url in enumerate(main_urls):
    category=categorys[c]
    gender_url=main_url+'20/761'    
    html_doc2=requests.get(gender_url).content.decode()
    soup2 = BeautifulSoup(html_doc2, 'html.parser')
    pages=soup2.find("div",attrs={"class":"paginationControl"})
    page_urls=[gender_url]
    if type(pages)==type(None):
        print('None type no pages')
        pass
    else:
        page_urls.append(gender_url+pages.find("a")['href'])
    print(page_urls)
    for page_url in page_urls:
        html_doc3=requests.get(page_url).content.decode()
        soup3 = BeautifulSoup(html_doc3, 'html.parser')
        for u in soup3.find_all("a",attrs={"class":"product-holder productIsActive"}):
            url='https://kidsbrandstore.se'+u['href']
            article_no=u['id']
            name=u.find("span",attrs={"class":"product-name-holder"}).text
            brand=u.find("span",attrs={"class":"product-brand-name-holder"}).text
            price=u.find("span",attrs={"class":"product-price"}).text
            img_url=u.find("img")['data-src']
            print(url)
            
            ac='kidsbrandstore'+'/'+article_no+str('.jpg')
            rr = requests.get(img_url, allow_redirects=True)
            open(ac, 'wb').write(rr.content)

            blob_name=str('kidsbrandstore')+str('/')+str(article_no)
            path_to_file='kidsbrandstore/'+article_no+str('.jpg')
            blob_url=upload_to_bucket(blob_name, path_to_file)

            
            
            html_doc3=requests.get(url).content.decode()
            soup3 = BeautifulSoup(html_doc3, 'html.parser')
            sizes=soup3.find("div",attrs={"id":"attribute-select-holder"})
            for sz in sizes.find_all("label"):
                size=sz.text
                with open("kidsbrandstore.csv", "a", encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow([name,brand,gender,category,size,price,url,img_url,article_no,blob_url])

        
        
    

None type no pages
['https://kidsbrandstore.se/byxor/20/761']
https://kidsbrandstore.se/under-armour-eu-cotton-fleece-jogger-35558
https://kidsbrandstore.se/peak-performance-jr-orig-pa-38399
https://kidsbrandstore.se/peak-performance-jr-orig-pa-38389
https://kidsbrandstore.se/gant-gant-logo-sweat-pants-35767
https://kidsbrandstore.se/gant-gant-logo-sweat-pants-35766
https://kidsbrandstore.se/scotch-soda-club-nomade-sweat-pant-38099
https://kidsbrandstore.se/scotch-soda-club-nomade-sweat-pant-38098
https://kidsbrandstore.se/calvin-klein-neon-tape-sweatpants-37105
https://kidsbrandstore.se/calvin-klein-side-stripe-punto-pants-35843
https://kidsbrandstore.se/tommy-hilfiger-essential-sweatpants-37074
https://kidsbrandstore.se/tommy-hilfiger-essential-sweatpants-37073
https://kidsbrandstore.se/henri-lloyd-marl-bb-joggers-37059
https://kidsbrandstore.se/henri-lloyd-bb-joggers-37061
https://kidsbrandstore.se/colmar-junior-pants-36159
https://kidsbrandstore.se/grunt-luppa-leggings-34646
https:

https://kidsbrandstore.se/tommy-hilfiger-essential-icon-knit-dress-slvls-33554
https://kidsbrandstore.se/ralph-lauren-pique-dress-dresses-woven-33108
https://kidsbrandstore.se/ralph-lauren-windowpane-dresses-woven-33109
https://kidsbrandstore.se/ralph-lauren-solid-ponte-dresses-knit-32971
https://kidsbrandstore.se/ralph-lauren-ruffle-dress-dresses-knit-32939
https://kidsbrandstore.se/designers-remix-nini-smock-dress-33940
https://kidsbrandstore.se/designers-remix-nini-wrap-dress-33936
https://kidsbrandstore.se/designers-remix-nini-wrap-dress-33935
https://kidsbrandstore.se/designers-remix-mirah-layer-dress-33930
https://kidsbrandstore.se/designers-remix-mirah-layer-dress-33929
https://kidsbrandstore.se/designers-remix-derrek-skirt-33928
https://kidsbrandstore.se/designers-remix-derrek-shirt-dress-33924
https://kidsbrandstore.se/designers-remix-babette-skirt-33920
https://kidsbrandstore.se/designers-remix-nini-skirt-33915
https://kidsbrandstore.se/designers-remix-nini-skirt-33914
https:

https://kidsbrandstore.se/garcia-girls-jacket-35650
https://kidsbrandstore.se/puma-puma-x-diamond-xt-track-top-38100
https://kidsbrandstore.se/ralph-lauren-bball-jacket-outerwear-denim-38102
https://kidsbrandstore.se/ralph-lauren-windbreaker-outerwear-jacket-35450
https://kidsbrandstore.se/ralph-lauren-windbreaker-outerwear-jacket-35451
https://kidsbrandstore.se/calvin-klein-recycled-light-down-bomber-35864
https://kidsbrandstore.se/calvin-klein-silver-light-hood-bomber-girl-35863
https://kidsbrandstore.se/calvin-klein-trucker-jkt-pageview-blue-com-37095
https://kidsbrandstore.se/franklin-marshall-mesh-windcheater-36187
https://kidsbrandstore.se/parajumpers-ugo-slw-36162
https://kidsbrandstore.se/colmar-boy-down-jacket-36100
https://kidsbrandstore.se/colmar-girl-down-jacket-35188
https://kidsbrandstore.se/colmar-girl-down-vest-35179
https://kidsbrandstore.se/colmar-girl-down-jacket-35177
https://kidsbrandstore.se/colmar-girl-down-vest-35166
https://kidsbrandstore.se/parajumpers-ugo-slw

https://kidsbrandstore.se/jack-wolfskin-heljar-3in1-jkt-31787
https://kidsbrandstore.se/jack-wolfskin-icefjord-jacket-31798
https://kidsbrandstore.se/jack-wolfskin-icefjord-jacket-31797
https://kidsbrandstore.se/lacoste-jacket-32468
https://kidsbrandstore.se/ralph-lauren-down-cap-jacket-32413
https://kidsbrandstore.se/ralph-lauren-quilted-jacket-32404
https://kidsbrandstore.se/peak-performance-frost-down-hoodie-jacket-32364
https://kidsbrandstore.se/ralph-lauren-hi-tech-jacket-32369
https://kidsbrandstore.se/ralph-lauren-down-cap-jacket-32378
https://kidsbrandstore.se/bergans-of-norway-knyken-insulated-jkt-girl-32070
https://kidsbrandstore.se/napapijri-k-skidoo-open-long-2-32314
https://kidsbrandstore.se/napapijri-k-skidoo-open-long-2-32315
None type no pages
['https://kidsbrandstore.se/regnklader/20/761']
https://kidsbrandstore.se/didriksons-fraenkel-bs-yt-coat-31890
https://kidsbrandstore.se/didriksons-thayer-gs-yt-coat-31248
https://kidsbrandstore.se/didriksons-thayer-gs-yt-coat-312

https://kidsbrandstore.se/tommy-hilfiger-basic-zip-hoodie-29948
https://kidsbrandstore.se/burton-girls-oak-po-34625
https://kidsbrandstore.se/burton-girls-clssc-mtn-po-34623
https://kidsbrandstore.se/8848-altitude-kaden-jr-sweat-35443
https://kidsbrandstore.se/8848-altitude-kaden-jr-sweat-34136
https://kidsbrandstore.se/8848-altitude-kaden-jr-sweat-35444
https://kidsbrandstore.se/8848-altitude-kaden-jr-sweat-35442
https://kidsbrandstore.se/ralph-lauren-bear-hoodie-tops-knit-35459
https://kidsbrandstore.se/diesel-sdivision-over-sweat-shirt-35400
https://kidsbrandstore.se/diesel-sdivision-over-sweat-shirt-35403
https://kidsbrandstore.se/didriksons-wien-gs-yt-sweater-34473
https://kidsbrandstore.se/didriksons-wien-gs-yt-sweater-34461
https://kidsbrandstore.se/hummel-moore-hoodie-35422
https://kidsbrandstore.se/tommy-hilfiger-knitted-tape-hoodie-33551
https://kidsbrandstore.se/diesel-salby-over-hood-32101
https://kidsbrandstore.se/tommy-hilfiger-knitted-tape-hoodie-33550
https://kidsbrands

https://kidsbrandstore.se/peak-performance-jr-ori-set-38393
https://kidsbrandstore.se/peak-performance-jr-orig-t-38392
https://kidsbrandstore.se/peak-performance-jr-orig-t-38398
https://kidsbrandstore.se/peak-performance-jr-orig-t-38397
https://kidsbrandstore.se/peak-performance-jr-origset-36533
https://kidsbrandstore.se/tommy-hilfiger-logo-embroidered-tee-ss-38128
https://kidsbrandstore.se/tommy-hilfiger-logo-embroidered-tee-ss-38126
https://kidsbrandstore.se/tommy-hilfiger-puff-print-tee-ss-38127
https://kidsbrandstore.se/gant-gant-collegiate-ss-t-shirt-35781
https://kidsbrandstore.se/gant-gant-collegiate-ss-t-shirt-35780
https://kidsbrandstore.se/gant-gant-collegiate-ss-t-shirt-35769
https://kidsbrandstore.se/gant-gant-1949-ss-t-shirt-35758
https://kidsbrandstore.se/gant-gant-1949-ss-t-shirt-35757
https://kidsbrandstore.se/gant-gant-collegiate-print-ss-t-sh-38097
https://kidsbrandstore.se/gant-gant-1949-ss-t-shirt-38091
https://kidsbrandstore.se/gant-gant-collegiate-print-ss-t-sh-38

https://kidsbrandstore.se/tommy-hilfiger-sport-flag-top-38182
https://kidsbrandstore.se/tommy-hilfiger-dg-erica-top-slvls-35984
https://kidsbrandstore.se/tommy-hilfiger-lace-tommy-tape-tee-ss-35972
https://kidsbrandstore.se/tommy-hilfiger-lace-tommy-tape-tee-ss-35971
https://kidsbrandstore.se/ralph-lauren-eyelet-top-tops-shirt-33124
https://kidsbrandstore.se/nike-studio-reversible-po-34594
https://kidsbrandstore.se/nike-studio-reversible-po-34593
https://kidsbrandstore.se/ralph-lauren-ruffle-yoke-tops-knit-32950
https://kidsbrandstore.se/ralph-lauren-ruffle-yoke-tops-knit-32949
https://kidsbrandstore.se/ralph-lauren-eyelet-top-tops-shirt-33123
https://kidsbrandstore.se/designers-remix-nini-blouse-33939
https://kidsbrandstore.se/designers-remix-nini-blouse-33938
https://kidsbrandstore.se/designers-remix-derrek-top-33927
https://kidsbrandstore.se/designers-remix-kikka-top-33923
https://kidsbrandstore.se/designers-remix-nini-top-33917
https://kidsbrandstore.se/designers-remix-nini-top-339

https://kidsbrandstore.se/tommy-hilfiger-tommy-vn-sweater-ls-35526
https://kidsbrandstore.se/ralph-lauren-ls-cn-tops-sweater-35462
https://kidsbrandstore.se/ralph-lauren-ls-cn-tops-sweater-35461
https://kidsbrandstore.se/diesel-screwdivision-over-sweat-shirt-35401
https://kidsbrandstore.se/hummel-dos-sweatshirt-35413
https://kidsbrandstore.se/hummel-tilda-zip-jacket-34206
https://kidsbrandstore.se/ralph-lauren-crewneck-tops-knit-35341
https://kidsbrandstore.se/gant-tb-gant-east-coast-c-neck-sweat-35282
https://kidsbrandstore.se/gant-tb-summer-logo-c-neck-sweat-35285
https://kidsbrandstore.se/gant-tb-summer-logo-c-neck-sweat-35281
https://kidsbrandstore.se/napapijri-k-bonthe-c-35251
https://kidsbrandstore.se/napapijri-k-bonthe-c-35263
https://kidsbrandstore.se/napapijri-k-bonthe-c-35257
https://kidsbrandstore.se/napapijri-k-byl-35259
https://kidsbrandstore.se/napapijri-k-byl-35258
https://kidsbrandstore.se/napapijri-k-byl-35255
https://kidsbrandstore.se/ralph-lauren-cable-cn-tops-sweate

https://kidsbrandstore.se/eastpak-padded-pakr-35153
https://kidsbrandstore.se/eastpak-padded-zipplr-35152
https://kidsbrandstore.se/eastpak-padded-zipplr-35150
https://kidsbrandstore.se/eastpak-padded-zipplr-35149
https://kidsbrandstore.se/fred-perry-retro-flight-bag-34602
https://kidsbrandstore.se/eastpak-padded-pakr-35156
https://kidsbrandstore.se/eastpak-padded-pakr-35155
https://kidsbrandstore.se/eastpak-padded-pakr-35148
https://kidsbrandstore.se/gant-d1-gym-bag-33376
https://kidsbrandstore.se/franklin-marshall-franklin-back-pack-30687
https://kidsbrandstore.se/kidsbrandstore-gympapase-29186
https://kidsbrandstore.se/gant-gant-original-bag-21329
https://kidsbrandstore.se/barbour-bintl-fairing-messenger-24032


In [8]:
'https://kidsbrandstore.se'+u['href']

'https://kidsbrandstore.se/barbour-bintl-fairing-messenger-24032'

In [9]:
u

<a class="product-holder productIsActive" href="/barbour-bintl-fairing-messenger-24032" id="product-entry-id-24032">
<span class="product-image">
<img alt="B.Intl Fairing Messenger" class="lazyload" data-src="https://d1zubn5gsgrqpe.cloudfront.net/images/product/thumb/barbour_vaskor-necessarer_svart_barnklader_24032-kjc8.jpg" src="data:image/gif;base64,R0lGODdhAQABAPAAAMPDwwAAACwAAAAAAQABAAACAkQBADs="/>
</span>
<div class="product-info">
<span class="product-brand-name-holder">Barbour</span>
<span class="product-name-holder">B.Intl Fairing Messenger</span>
<span class="product-price">
<span class="discountPrice">520 SEK</span><span class="oldPrice">1 299 SEK</span> </span>
</div>
<!--             <div class="product-btn">Till produkten</div> -->
</a>

In [10]:
u['id']

'product-entry-id-24032'

In [11]:
u.find("span",attrs={"class":"product-name-holder"}).text

'B.Intl Fairing Messenger'

In [12]:
u.find("span",attrs={"class":"product-brand-name-holder"}).text

'Barbour'

In [13]:
u.find("span",attrs={"class":"product-price"}).text

'\n520 SEK1 299 SEK '

In [14]:
u.find("img")['data-src']

'https://d1zubn5gsgrqpe.cloudfront.net/images/product/thumb/barbour_vaskor-necessarer_svart_barnklader_24032-kjc8.jpg'

In [15]:
u.find("img")['alt']

'B.Intl Fairing Messenger'

In [16]:
if type(pages)==type(None):
    print('None type no pages')
    pass

None type no pages


In [17]:
type(pages)==type(None)

True

In [18]:
from google.cloud import storage
storage_client = storage.Client.from_service_account_json('auth.json')
bucket = storage_client.get_bucket('urbankiddo')
blob = bucket.blob('kidsbrandstore.csv')
blob.upload_from_filename('kidsbrandstore.csv')
blob.public_url

'https://storage.googleapis.com/urbankiddo/kidsbrandstore.csv'